## Identify WDPAs in each gadm1


This notebook performs a spatial join between the WDPAs dataset and the gadm1 administrative boundaries to identify which WDPAs belong to each of the regions defined in the gadm1.

#### Import packages

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
import requests as re
from copy import deepcopy

#### Connect to arcgis online with credentials saved in .env file

In [76]:
env_path = ".env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

aol_password = env['aol_key']
aol_username = env['aol_username']

gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)

#### Read files

In [143]:
## read tables saved in local as geodataframes
wdpa = gpd.read_file('zip:///Users/sofia/Documents/HE_Data/WDPA_FILTERED_20210615_FILTERED_nomarine_with_oecm_wdpa.zip')
gadm1 = gpd.read_file('zip:///Users/sofia/Documents/HE_Data/gadm36_level1.zip')

In [138]:
# ## read gadm1 calling it from arcgis online
# item_id = '3f89b653de41428a8166d51f383f0950'
# item = gis.content.get(item_id)
# flayer = item.layers[0]
# gadm1 = pd.DataFrame.spatial.from_layer(flayer) # this is a sdf

In [139]:
# ## convert gadm1 from dataframe to geodataframe
# gadm1 = gpd.GeoDataFrame(gadm1, geometry='SHAPE')
# gadm1.crs
# gadm1 = gadm1.set_crs(epsg=3857) # set crs
# gadm1 = gadm1.to_crs(epsg=4326) # project to wdpa crs

#### Select only relevant columns

In [144]:
gadm = gadm1[['GID_0','NAME_0','NAME_1','MOL_ID','geometry']]
gadm.rename(columns={'MOL_ID':'MOL_IDg'},inplace=True)
wd = wdpa[['MOL_ID','ISO3','ORIG_NA','STATUS','GOV_TYP','DESIG_E','DESIG_T','AREA_KM','geometry']]

/opt/anaconda3/envs/vizz38/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


#### Spatial join

In [141]:
gwd = gpd.sjoin(gadm,wd, how='right')

#### Save file as shp and as csv

In [73]:
gwd = gwd.drop(columns=['index_left'])
gwd.to_file(filename='/Users/sofia/Documents/HE_Data/wdpa_with_gadm1', driver='ESRI Shapefile')
gwd = gwd.drop(columns=['geometry'])
gwd.to_csv('/Users/sofia/Documents/HE_Data/wdpa_with_gadm1.csv')

#### Publish layers with API

In [145]:
# Function to publish files as feature services or hosted tables

def publishCSVasFS(csv_file,csvName, gis, aol_folder_name, sharing, table): # sharing = 'None', 'groups', 'everyone' # table = 'yes', 'no'
    try:
        
        if gis.content.is_service_name_available(csvName, "featureService"):
            print(f"Service name {csvName} is available")
            #csv_file = f'./{csvName}.csv'
            csv_item = gis.content.add({}, csv_file)
            #csv_lyr = csv_item.publish(None,  {'CountryCode' : 'countryname'})
            if table == 'yes':
                csv_lyr = csv_item.publish(publish_parameters={'locationType':'none', 'type':'csv'})
            else:
                csv_lyr = csv_item.publish()
                flayer_collection = FeatureLayerCollection.fromitem(csv_lyr)
                searched_flayer = flayer_collection.layers[0] 
                nber_features = searched_flayer.query(return_count_only=True)
                print(f"The service {csvName} has been published. The service has {nber_features} entries")
            print(f"Moving service {csvName} to {aol_folder_name} in ArcGIS Online...")
            # move both the csv_item and csv_lyr items into this new folder
            csv_item.move(aol_folder_name)
            csv_lyr.move(aol_folder_name)
            print(f"Service {csvName} has been moved to {aol_folder_name} in ArcGIS Online")
#             sharing, default is owner
#             csv_lyr.shared_with
            if sharing == 'None':
                csv_lyr.share(everyone=False, org=False, groups=None, allow_members_to_edit=False)
            if sharing == 'groups':
            # the group id is needed to share with agroup
                csv_lyr.share(everyone=False, org=False, groups=[group_id], allow_members_to_edit=False)
            if sharing == 'everyone':
                csv_lyr.share(everyone=True, org=False, groups=None, allow_members_to_edit=False)
            sharing_prop = csv_lyr.shared_with
            if sharing_prop['everyone']==True:
                print(f"shared with everyone")   
            else:
                print(f"not a public layer, for this layer to be used it has to be public or the urls have to be whitelisted")            
            #not allowing deleting
            csv_lyr.protect()
            print(f"{csvName}'s protection against deletion : {csv_lyr.protected}") 
            # it is possible to check the status with csv_item.content_status
            return csv_lyr.id
        else:
            print("The service name is not available, try overwritting, appending the data or a different service name")
    except:
        print("The csv hasn't been published")

In [148]:
# Run function with the file we want to add
csv_file = '/Users/sofia/Documents/HE_Data/wdpa_with_gadm1.zip'
csvName = 'wdpa_with_gadm1_2'
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)
aol_folder_name = '#6 precalculated data'
group_id='46783c33c294485abe83f5e899c2f21f'
publishCSVasFS(csv_file,csvName,gis,aol_folder_name,sharing='groups')

TypeError: publishCSVasFS() missing 1 required positional argument: 'table'